## Study Visualizations

### Import Packages
The Python packages required by this notebook are imported. The package and Python interpreter versions are printed to ensure analysis reproducibility.

In [1]:
import os
import sys
import pandas
import plotly
import plotly.express as px
import plotly.graph_objects as go
import IPython
from collections import OrderedDict
print(f"Python version: {sys.version}.")
print(f"IPython package version: {IPython.__version__}.")
print(f"pandas package version: {pandas.__version__}.")
print(f"plotly package version: {plotly.__version__}.")

Python version: 3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0].
IPython package version: 7.22.0.
pandas package version: 1.2.4.
plotly package version: 4.14.3.


### Define and Load Data Sources
The data files from the case specific `postProcessing` folder are loaded into pandas dataframes.

In [2]:
# Define data file information
# data_file_info = OrderedDict()
data_file_info = {'inlet1_massflow':["Time [s]", "inlet1 Mass Flow [kg/s]"],
                  'inlet2_massflow':["Time [s]", "inlet2 Mass Flow [kg/s]"],
                  'outlet_massflow':["Time [s]", "outlet Mass Flow [kg/s]"],
                  'outlet_temperature':["Time [s]", "outlet Average Temperature [K]"],
                  'residuals':["Time [s]", "Ux", "Uy","Uz","p_rgh","e", "k", "epsilon"],
                  'pressure_drop':["Time [s]", "inlet1-outlet Pressure Drop [Pa]"]}

# Create simulation data dictionary
# simulation_data = OrderedDict()
simulation_data = {}
# Create case list
case_list = []
# Traverse simulation repository, po;ulate case list and load case data
for root, dirs, files in os.walk("../", topdown=True):
    for name in files:
        for data_source, source_headers in data_file_info.items():
            file_path = os.path.join(root, name)
            file_match = '/postProcessing/' + str(data_source) + '/'
            if file_match in file_path:
                case_name = file_path.split('/')[1]
                if case_name not in case_list:
                    case_list.append(case_name)
                case_id = case_name + ':' + data_source
                data_source_data = pandas.read_csv(file_path, delim_whitespace=True, comment ='#',
                                   header = None, names = source_headers, float_precision='round_trip')
                simulation_data[case_id] = data_source_data


### Create Plots
Create interactive plots for the simulation data.

In [3]:
# Set figure dimensions for inline display
figure_width = 1000
figure_height = 900
# Initialize figure number
figure_number = 1
# Initialize figure list
figure_list = []
for data_key, data_value in simulation_data.items():
    # Set figure title
    case_name = data_key.split(':')[0]
    file_name = data_key.split(':')[1]
    figure_title = "Figure " + str(figure_number) + ": Case " + case_name + " - " + file_name
    # Get data into a dataframe
    data_df = data_value
    # Get x axis value
    x_val = data_df['Time [s]']
    # Define figure
    fig = go.Figure()
    # Iterate over dataframe parameter columns
    for column in data_df.columns[1:]:
        # Get y axis value
        y_val = data_df[column]
        fig.add_trace(
            go.Scatter(
                x = x_val,
                y = y_val,
                mode = 'lines',
                name = column)
        )

    if file_name == 'residuals':
        yaxis_type_value = 'log'
        yaxis_title_value = 'Residuals'
    else:
        yaxis_type_value = 'linear'
        yaxis_title_value = column
        
    fig.update_layout(
        title_text=figure_title,
        title_x=0.5,
        xaxis_title=data_df.columns[0],
        yaxis_title=yaxis_title_value
    )
    
    fig.update_yaxes(
        showexponent='all',
        exponentformat='e',
        minexponent=5,
        type=yaxis_type_value
    )

    figure_html_name = str(figure_number) + "-" + case_name + "-" + file_name + ".html"
    figure_list.append(fig)
    figure_number += 1

# Write the all simulation plots to a single html file
plot_package = 'flow-header-plots.html'
with open(plot_package, 'w') as f:
    for figure in figure_list:
        f.write(figure.to_html(full_html=False, include_plotlyjs='cdn'))

# Display simulation plots
IPython.display.IFrame(plot_package, width=figure_width, height=figure_height)

In [4]:
# Export notebook to html
os.system('jupyter nbconvert --to html flow-header.ipynb')

0